In [24]:
import ROOT as rt
# import root_numpy as rtnp
import csv
import re
import sys
import collections

from collections import OrderedDict
import uproot
import pandas as pd

import scipy
import awkward
import numpy as np
import time
import numba
from numba import jit
from matplotlib import pyplot as plt
sys.path.append('/storage/af/user/christiw/gpu/christiw/llp/delayed_jet_analyzer/lib/')
from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot
from scipy.stats import norm

import CMS_lumi, tdrstyle
a = tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 0



# donotdelete = []
print(sys.version)

3.6.8 (default, Aug  7 2019, 17:28:10) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-39)]


# Load ntuples

In [25]:
fpath_bkg =OrderedDict()
tree_bkg = OrderedDict()
tree_sig = OrderedDict()
fpath_sig =OrderedDict()


start_t = time.time()



isData = 1
if isData:
    ntupler_version = 'V1p17/'
    
    analyzer_version = "/v5/v130/"


    data_path = '/storage/af/user/christiw/login-1/christiw/LLP/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/Data2018/'+analyzer_version+'/normalized/'
    
    fpath_bkg['data'] = data_path + 'Run2_displacedJetMuonNtupler_V1p17_Data2016_Data2017_Data2018-HighMET_goodLumi.root'
#     fpath_bkg['data18'] = data_path + 'Run2_displacedJetMuonNtupler_V1p17_Data2018_17Sept2018_Run2018-HighMET-17Sep2018_goodLumi.root'
#     data_path = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/Data2017/'+analyzer_version+'/normalized/'
#     fpath_bkg['data17'] = data_path + 'Run2_displacedJetMuonNtupler_V1p17_Data2017_Run2017-HighMET-17Nov2017_goodLumi.root'
#     data_path = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/Data2016/'+analyzer_version+'/normalized/'
#     fpath_bkg['data16'] = data_path + 'Run2_displacedJetMuonNtupler_V1p17_Data2016_Run2016-HighMET-07Aug17_goodLumi.root'



    

else:
    ntupler_version = 'V1p17/'

    analyzer_version = "/v1/v124/" #includes DT
    #path = ' /storage/cms/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc//V1p17/'
    path = '/storage/af/user/christiw/login-1/christiw/LLP/displacedJetMuonAnalyzer/csc/'


    decay = 'bbbb'
    mass = '55'
    lumi = [ 35920, 41530, 59740 ]
    years = ['MC_Summer16', 'MC_Fall17', 'MC_Fall18']
    tune = ['TuneCUETP8M1_13TeV-powheg-pythia8','TuneCP5_13TeV-powheg-pythia8','TuneCP5_13TeV-powheg-pythia8']
    ctaus  = ['100','1000','10000','100000']
    year_index = 0
#     for ct in ctaus:
        
#         if year_index == 3:
#             mc_path = path+ntupler_version+'/MC_all/'+analyzer_version+'/normalized/'
#             fpath_bkg['m'+mass+'ctau'+ct] = mc_path + 'ggH_HToSSTo'+decay+'_MH-125_MS-'+str(mass)+'_ctau-'+ct+'_137000pb_weighted.root'
#         else:
#             mc_path = path+ntupler_version+'/'+years[year_index]+'/'+analyzer_version+'/normalized/'
#             fpath_bkg['m'+mass+'ctau'+ct] = mc_path + 'ggH_HToSSTo'+decay+'_MH-125_MS-'+str(mass)+'_ctau-'+ct+'_'+tune[year_index]+'_'+str(lumi[year_index])+'pb_weighted.root'

            
    fpath_bkg['test'] = '/storage/af/user/christiw/login-1/christiw/LLP/displacedJetMuonAnalyzer/privateProduction/RunIIFall18/v1/v1/normalized/ggH_HToSSTobbbb_ms55_pl1000_1pb_weighted.root'
NEvents = {}
Total = {}
accep = {}
accep_met = {}
for k,v in fpath_bkg.items():
    print (k, v)
    root_dir = uproot.open(v) 
    tree_bkg[k] = root_dir['MuonSystem']
    NEvents[k] = root_dir['NEvents'][1]
    if not isData:
        Total[k] = root_dir['Total'][1]
        accep[k] = root_dir['acceptance'][1]
        accep_met[k] = root_dir['acceptance_met'][1]

    a = tree_bkg[k]["weight"].array()
    print("NEvents",NEvents[k],len(a))


data /storage/af/user/christiw/login-1/christiw/LLP/displacedJetMuonAnalyzer/csc/V1p17//Data2018//v5/v130//normalized/Run2_displacedJetMuonNtupler_V1p17_Data2016_Data2017_Data2018-HighMET_goodLumi.root
NEvents 22763250.0 21048829


# background cut flow

In [27]:
JET_PT_CUT = 20.0
MUON_PT_CUT = 10.0
N_RECHIT_CUT = 100
jetPt_cut = 30

weight = {}
sel_ev = {}

ncluster = 1
cluster_index = '3'
table = OrderedDict()
table['acc'] = "Acceptance "
table['met'] = "Trigger and MET cut "
table['met_filter'] = "MET filters "



table['lep'] = "$N_{lepton} = 0$ "
table['jet'] = r"$N_{jet} \geq 1$ "
table['dphi_jetmet'] = 'dphi(jet, met)'
table['dt_station'] = 'dt_station<3'
table['dt_wheel'] = 'dt_wheel<3'

table['1_cluster'] = "$N_{cluster} \geq 1$ "
table['jet_veto'] = "jet veto "
table['muon_veto'] = "muon veto "
table['mb1'] = "MB1 veto "
table['rpc'] = "rpc matching "
table['mb1_adjacent'] = "mb1_adjacent "
table['dphi'] = r"$\Delta\phi\mathrm{ (cluster,MET)}$ "
table['nrechits'] = r"$N_{rechits}$ cut "

for k, T in tree_bkg.items():
    if not k == 'data':continue

    ########### SELECTION: JETS ############

    sel_jet = np.logical_and(T.array('jetPt') >= jetPt_cut, np.abs(T.array('jetEta')) < 2.4 )
    sel_jet = np.logical_and(sel_jet, np.abs(T.array('jetTightPassId')))


    ########### SELECTION: EVENTS ############
    hlt = T['HLTDecision'].array()


    total = NEvents[k]


    sel_ev[k] = T.array('METNoMuTrigger')
    sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('metEENoise')>=200)

    accep_met[k] = np.count_nonzero([sel_ev[k]])
    print("trigger + MET", accep_met[k], np.count_nonzero([sel_ev[k]])/total, accep_met[k]/total,np.count_nonzero(sel_ev[k]))
    table["met"] += " & {0:6.3f} & {1:6.3f} & {2:6.0f}".format(100*accep_met[k]/total, 100*accep_met[k]/total, np.count_nonzero([sel_ev[k]]))






    acc_met = np.count_nonzero([sel_ev[k]])
    sel_temp = np.count_nonzero([sel_ev[k]])


    sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('Flag2_all') == 1)
    print("MET filters", np.count_nonzero([sel_ev[k]])/acc_met, np.count_nonzero([sel_ev[k]])/sel_temp,np.count_nonzero(sel_ev[k]))
    table["met_filter"] += " & {0:6.2f} & {1:6.2f} & {2:6.0f}".format(100*np.count_nonzero([sel_ev[k]])/sel_temp, 100*np.count_nonzero([sel_ev[k]])/acc_met, np.count_nonzero([sel_ev[k]]))



    sel_temp = np.count_nonzero([sel_ev[k]])


    sel_ev[k] = np.logical_and(sel_ev[k] , sel_jet.sum()>=1)
    print("1 jet", np.count_nonzero(sel_ev[k])/acc_met, np.count_nonzero(sel_ev[k])/sel_temp,np.count_nonzero(sel_ev[k]))
    table["jet"] += " & {0:6.2f} & {1:6.2f} & {2:6.0f}".format(100*np.count_nonzero([sel_ev[k]])/sel_temp, 100*np.count_nonzero([sel_ev[k]])/acc_met, np.count_nonzero([sel_ev[k]]))
    sel_temp = np.count_nonzero([sel_ev[k]])

    sel_ev[k] = np.logical_and(sel_ev[k] , np.abs(T.array('jetMet_dPhiMin'))>0.6)
    print("dphi(jet, met)",np.count_nonzero(sel_ev[k])/acc_met, np.count_nonzero(sel_ev[k])/sel_temp,np.count_nonzero(sel_ev[k]))
    table["dphi_jetmet"] += " & {0:6.2f} & {1:6.2f} & {2:6.0f}".format(100*np.count_nonzero([sel_ev[k]])/sel_temp, 100*np.count_nonzero([sel_ev[k]])/acc_met, np.count_nonzero([sel_ev[k]]))
    sel_temp = np.count_nonzero([sel_ev[k]])
    # cosmic muon veto
    sel_ev[k] = np.logical_and(sel_ev[k] , T.array("nDtStations25") < 3)
    print("DT station < 3", np.count_nonzero(sel_ev[k])/acc_met, np.count_nonzero(sel_ev[k])/sel_temp,np.count_nonzero(sel_ev[k]))
    table["dt_station"] += " & {0:6.2f} & {1:6.2f} & {2:6.0f}".format(100*np.count_nonzero([sel_ev[k]])/sel_temp, 100*np.count_nonzero([sel_ev[k]])/acc_met, np.count_nonzero([sel_ev[k]]))
    sel_temp = np.count_nonzero([sel_ev[k]])

    sel_ev[k] = np.logical_and(sel_ev[k] , T.array("nDtWheels25") < 3)
    print("DT wheel < 3", np.count_nonzero(sel_ev[k])/acc_met, np.count_nonzero(sel_ev[k])/sel_temp,np.count_nonzero(sel_ev[k]))
    table["dt_wheel"] += " & {0:6.2f} & {1:6.2f} & {2:6.0f}".format(100*np.count_nonzero([sel_ev[k]])/sel_temp, 100*np.count_nonzero([sel_ev[k]])/acc_met, np.count_nonzero([sel_ev[k]]))

    sel_temp = np.count_nonzero([sel_ev[k]])


    ########### SELECTION: CLUSTERS ############

    
    
    binD = np.logical_and(T.array('dtRechitClusterSize')>=100, np.abs(T.array('dtRechitClusterMetEENoise_dPhi'))<1)
#     binD = T.array('dtRechitClusterSize')<0 #for unblinding


#     temp = T.array('dtRechitCluster_match_RPCBx_dPhi0p5')
#     time = np.array([np.array([scipy.stats.mode(y)[0] for y in x]) for x in temp])

   
    
    
    sel_rechitcluster = np.abs(T.array('dtRechitClusterX'))>=0
    #signal region blind D
    sel_rechitcluster = np.logical_and(sel_rechitcluster, np.logical_not(binD))
    
    #oot
#     sel_rechitcluster = np.logical_and(sel_rechitcluster, awkward.fromiter(time)<0)
#     sel_rechitcluster = sel_rechitcluster.any()

    
    print("cluster efficiency", np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/acc_met, np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/sel_temp)
    table["1_cluster"] +=  " & {0:6.2f} & {1:6.2f} & {2:6.0f}".format(100*np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/sel_temp,\
                                                      100*np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/acc_met, np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)))  

    sel_temp = np.count_nonzero([np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])




    sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('dtRechitClusterJetVetoPt') < JET_PT_CUT)
    print("jet cut", np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/acc_met, np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/sel_temp)
    table["jet_veto"] +=   " & {0:6.2f} & {1:6.2f} & {2:6.0f}".format(100*np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/sel_temp,\
                                                      100*np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/acc_met, np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)))  
    sel_temp = np.count_nonzero([np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])
    
    
    sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('dtRechitClusterMuonVetoPt') < MUON_PT_CUT)
    print("muon cut", np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/acc_met, np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/sel_temp)
    table["muon_veto"] +=  " & {0:6.2f} & {1:6.2f} & {2:6.0f}".format(100*np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/sel_temp,\
                                                      100*np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/acc_met, np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)))  
    sel_temp = np.count_nonzero([np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])



    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster_match_MB1hits_0p5') <= 1)
    print("mb1", np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/acc_met, np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/sel_temp)
    table["mb1"] +=  " & {0:6.2f} & {1:6.5f} & {2:6.0f}".format(100*np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/sel_temp,\
                                                      100*np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/acc_met, np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)))  
    sel_temp = np.count_nonzero([np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])



    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster_match_RPChits_dPhi0p5') > 0)
    print("rpc matching", np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/acc_met, np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/sel_temp)
    table["rpc"] +=  " & {0:6.2f} & {1:6.5f} & {2:6.0f}".format(100*np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/sel_temp,\
                                                      100*np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/acc_met, np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)))  
    sel_temp = np.count_nonzero([np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])


    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster_match_MB1hits_cosmics_minus') <= 8)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster_match_MB1hits_cosmics_plus') <= 8)

    print("mb1 adjacent", np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/acc_met, np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/sel_temp)
    table["mb1_adjacent"] +=  " & {0:6.2f} & {1:6.5f} & {2:6.0f}".format(100*np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/sel_temp,\
                                                      100*np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/acc_met, np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)))  
    sel_temp = np.count_nonzero([np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])

    
    
    
    
    
    sel_rechitcluster = np.logical_and(sel_rechitcluster, np.abs(T.array('dtRechitClusterMetEENoise_dPhi')) < 1)
    print("dphi(cluster, met) cut", np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/acc_met, np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/sel_temp)
    table["dphi"] +=   " & {0:6.2f} & {1:6.5f} & {2:6.0f}".format(100*np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/sel_temp,\
                                                      100*np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/acc_met, np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)))  
    sel_temp = np.count_nonzero([np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])


    sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('dtRechitClusterSize')>=100)

    print("N rechits", np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/acc_met, np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/sel_temp)
    table["nrechits"] +=  " & {0:6.2f} & {1:6.5f} & {2:6.0f}".format(100*np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/sel_temp,\
                                                      100*np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1))/acc_met, np.count_nonzero(np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)))  
    sel_temp = np.count_nonzero([np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])



for k,v in table.items():
    print(v+r" \\")


trigger + MET 14129474 0.6207142653180016 0.6207142653180016 14129474
MET filters 0.9261875565926941 0.9261875565926941 13086543
1 jet 0.8087214711602144 0.8731724642634804 11426809
dphi(jet, met) 0.37192332849757886 0.45989050836502127 5255081
DT station < 3 0.34078126333648373 0.9162675132885678 4815060
DT wheel < 3 0.2891639844483949 0.8485325208823982 4085735
cluster efficiency 0.005446133380478282 0.018834065351766575
jet cut 0.0007284064502330377 0.1337474496757677
muon cut 0.0005291775192763723 0.7264865915273999
mb1 1.139462091794783e-05 0.02153270028086131
rpc matching 5.024957050772025e-06 0.4409937888198758
mb1 adjacent 4.954183007803405e-06 0.9859154929577465
dphi(cluster, met) cut 1.4154808593724012e-06 0.2857142857142857
N rechits 0.0 0.0
Acceptance  \\
Trigger and MET cut  & 62.071 & 62.071 & 14129474 \\
MET filters  &  92.62 &  92.62 & 13086543 \\
$N_{lepton} = 0$  \\
$N_{jet} \geq 1$  &  87.32 &  80.87 & 11426809 \\
dphi(jet, met) &  45.99 &  37.19 & 5255081 \\
dt_stat

# signal cut flow

In [17]:
JET_PT_CUT = 20.0
MUON_PT_CUT = 10.0
N_RECHIT_CUT = 100
jetPt_cut = 30

weight = {}
sel_ev = {}

ncluster = 1
cluster_index = '3'
table = OrderedDict()
table['acc'] = "Acceptance "
table['met'] = "Trigger and MET cut "
table['met_filter'] = "MET filters "



table['lep'] = "$N_{lepton} = 0$ "
table['jet'] = r"$N_{jet} \geq 1$ "
table['dphi_jetmet'] = 'dphi(jet, met)'
table['dt_station'] = 'dt_station<3'
table['dt_wheel'] = 'dt_wheel<3'

table['1_cluster'] = "$N_{cluster} \geq 1$ "
table['jet_veto'] = "jet veto "
table['muon_veto'] = "muon veto "
table['mb1'] = "MB1 veto "
table['rpc'] = "rpc matching "
table['mb1_adjacent'] = "mb1_adjacent "
table['dphi'] = r"$\Delta\phi\mathrm{ (cluster,MET)}$ "
table['nrechits'] = r"$N_{rechits}$ cut "
table['mb2'] = 'mb2'
for k, T in tree_bkg.items():
    if '100000' in k:continue
#     if not k == 'm15ctau1000':continue
    weight = T.array('weight')*T.array('pileupWeight')*T.array('higgsPtWeight')*T.array('metSF')

    weight = T.array('weight')*T.array('pileupWeight')
    weight = T.array('weight')
    print(weight[:10])

    ########### SELECTION: JETS ############

    sel_jet = np.logical_and(T.array('jetPt') >= jetPt_cut, np.abs(T.array('jetEta')) < 2.4 )

    ########### SELECTION: EVENTS ############
    hlt = T['HLTDecision'].array()

#     sel_ev[k] = np.sum(T.array('gLLP_csc'),axis = 1) >= 1
#     acceptance = np.sum((T.array('weight')*T.array('higgsPtWeight'))[sel_ev[k]])

    total = np.sum(weight)
#     total = NEvents[k]
#     print("acceptance", acceptance/total, np.count_nonzero(sel_ev[k]))
#     table["acc"] += " & {0:6.3f} & {1:6.3f}".format(100*acceptance/total,100*acceptance/total)
#     sel_temp = np.sum(T.array('weight')[sel_ev[k]])

    sel_ev[k] = T.array('METNoMuTrigger')
#     sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('metEENoise')>=200)
    sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('met')>=200)
    sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('Flag2_all') == 1)


    accep_met[k] = np.sum(weight[sel_ev[k]])
    print("trigger + MET", accep_met[k], np.sum(weight[sel_ev[k]])/total, accep_met[k]/total,np.count_nonzero(sel_ev[k]))
    table["met"] += " & {0:6.3f} & {1:6.3f}".format(100*accep_met[k]/total, 100*accep_met[k]/total)






    acc_met = np.sum(weight[sel_ev[k]])
    sel_temp = np.sum(weight[sel_ev[k]])


    print("MET filters", np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]])/sel_temp,np.count_nonzero(sel_ev[k]))
    table["met_filter"] += " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp, 100*np.sum(weight[sel_ev[k]])/acc_met)



    sel_temp = np.sum(weight[sel_ev[k]])


    sel_ev[k] = np.logical_and(sel_ev[k] , sel_jet.sum()>=1)
    print("1 jet", np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]])/sel_temp,np.count_nonzero(sel_ev[k]))
    table["jet"] += " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp, 100*np.sum(weight[sel_ev[k]])/acc_met)
    sel_temp = np.sum(weight[sel_ev[k]])

    sel_ev[k] = np.logical_and(sel_ev[k] , np.abs(T.array('jetMet_dPhiMin'))>0.6)
    print("dphi(jet, met)", np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]])/sel_temp,np.count_nonzero(sel_ev[k]))
    table["dphi_jetmet"] += " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp, 100*np.sum(weight[sel_ev[k]])/acc_met)
    sel_temp = np.sum(weight[sel_ev[k]])
    # cosmic muon veto
    sel_ev[k] = np.logical_and(sel_ev[k] , T.array("nDtStations25") < 3)
    print("DT station < 3", np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]])/sel_temp,np.count_nonzero(sel_ev[k]))
    table["dt_station"] += " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp, 100*np.sum(weight[sel_ev[k]])/acc_met)
    sel_temp = np.sum(weight[sel_ev[k]])

    sel_ev[k] = np.logical_and(sel_ev[k] , T.array("nDtWheels25") < 3)
    print("DT wheel < 3", np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]])/sel_temp,np.count_nonzero(sel_ev[k]))
    table["dt_wheel"] += " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp, 100*np.sum(weight[sel_ev[k]])/acc_met)

    sel_temp = np.sum(weight[sel_ev[k]])


    ########### SELECTION: CLUSTERS ############

    sel_rechitcluster = T.array('dtRechitCluster_match_gLLP')
    print("cluster efficiency", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["1_cluster"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)  

    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])




    sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('dtRechitClusterJetVetoPt') < 20)
    print("jet cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["jet_veto"] +=   " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)       
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])
    
    
    sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('dtRechitClusterMuonVetoPt') < 10)
    print("muon cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["muon_veto"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)   
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])



#     sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster_match_MB1Seg_0p5') <= 0)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster_match_MB1hits_0p5') <= 1)


    print("mb1", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["mb1"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)   
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])



    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster_match_RPChits_dPhi0p5') > 0)
    print("rpc matching", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["rpc"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)   
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])


    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster_match_MB1hits_cosmics_minus') <= 8)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster_match_MB1hits_cosmics_plus') <= 8)

    print("mb1 adjacent", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["mb1_adjacent"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)     
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])

    
    
    
    
    
    sel_rechitcluster = np.logical_and(sel_rechitcluster, np.abs(T.array('dtRechitClusterMetEENoise_dPhi')) < 1)
#     sel_rechitcluster = np.logical_and(sel_rechitcluster, np.abs(T.array('dtRechitClusterMet_dPhi')) < 1)
    print("dphi(cluster, met) cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["dphi"] +=   " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)     
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])


    sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('dtRechitClusterSize')>=100)

    print("N rechits", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["nrechits"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)        
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])

    sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('dtRechitClusterMaxStation')==2)

    print("MB2", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["mb2"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)        
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])

for k,v in table.items():
    print(v+r" \\")


[0.9055811 0.9055811 0.9055811 0.9055811 0.9055811 0.9055811 0.9055811
 0.9055811 0.9055811 0.9055811]
trigger + MET 127.68692 7.31734e-05 7.31734e-05 141
MET filters 1.0 1.0 141
1 jet 0.9929078 0.9929078 140
dphi(jet, met) 0.09219861 0.09285717 13
DT station < 3 0.09219861 1.0 13
DT wheel < 3 0.09219861 1.0 13
cluster efficiency 0.0 0.0


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:120: RuntimeWarning: invalid value encountered in float_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:121: RuntimeWarning: invalid value encountered in double_scalars


jet cut 0.0 nan


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:127: RuntimeWarning: invalid value encountered in float_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:128: RuntimeWarning: invalid value encountered in double_scalars


muon cut 0.0 nan


...

dphi(cluster, met) cut 0.0 nan


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:175: RuntimeWarning: invalid value encountered in float_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:176: RuntimeWarning: invalid value encountered in double_scalars


N rechits 0.0 nan


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:182: RuntimeWarning: invalid value encountered in float_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:183: RuntimeWarning: invalid value encountered in double_scalars


MB2 0.0 nan
[0.859632 0.859632 0.859632 0.859632 0.859632 0.859632 0.859632 0.859632
 0.859632 0.859632]
trigger + MET 6171.918 0.0035369347 0.0035369347 7181
MET filters 1.0 1.0 7181
1 jet 0.9891375 0.9891375 7103
dphi(jet, met) 0.5046687 0.5102109 3624
DT station < 3 0.44200325 0.8758285 3174
DT wheel < 3 0.43587488 0.98613507 3130
cluster efficiency 0.12017184 0.2757026
jet cut 0.11195424 0.931618
muon cut 0.08284865 0.7400224
mb1 0.02924072 0.35294142
rpc matching 0.029101439 0.99523675
mb1 adjacent 0.028127864 0.9665455
dphi(cluster, met) cut 0.028127864 1.0
N rechits 0.024924396 0.8861105
MB2 0.006685497 0.26823103
[0.8776931 0.8776931 0.8776931 0.8776931 0.8776931 0.8776931 0.8776931
 0.8776931 0.8776931 0.8776931]
trigger + MET 22247.818 0.01274953 0.01274953 25352
MET filters 1.0 1.0 25352
1 jet 0.9775936 0.9775936 24784
dphi(jet, met) 0.8666362 0.8864995 21971
DT station < 3 0.84809595 0.97860664 21501
DT wheel < 3 0.8458478 0.99734914 21444
cluster efficiency 0.11852819 0.14